In [21]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [22]:
import os
import re
import time

import pandas as pd
import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.notebook import tqdm

<IPython.core.display.Javascript object>

In [23]:
def browser(url):
    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    driver = webdriver.Chrome(executable_path='/Users/admin/Git/web-scraping/OneFootbal/chromedriver',options=option)
    driver.get(url)
    return driver


def get_html_full(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-competition-results-stream/section/of-entity-page-root/div/of-xpa-switch-entity-deprecated[7]/of-simple-match-cards-list-deprecated/div/button"
    )
    el.click()
    time.sleep(5)
    page = driver.page_source
    driver.close()
    return BeautifulSoup(page)


def get_html_full_match(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-competition-results-stream/section/of-entity-page-root/div/of-xpa-switch-entity-deprecated[7]/of-simple-match-cards-list-deprecated/div/button"
    )
    el.click()
    time.sleep(5)
    page = driver.page_source
    driver.close()
    return BeautifulSoup(page)


def get_html_from_a_match(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-match-stream-v2/section/of-xpa-layout-match/section[2]/of-xpa-switch-match/of-match-events/div/button"
    )
    el.click()
    time.sleep(2)
    page = driver.page_source
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-match-stream-v2/section/of-xpa-layout-match/section[3]/div[1]/div/of-xpa-switch-match/of-match-lineup-v2/section/nav/ul/li[2]/button"
    )
    el.click()
    time.sleep(2)
    second_lineup = driver.page_source
    driver.close()
    return BeautifulSoup(page), BeautifulSoup(second_lineup)



def format_field(string):
    string = str(string)
    string = string.lower()
    string = string.strip()
    string = string.replace("  ", " ")
    string = string.replace(" ", "_")
    return string


def get_info_from_event(event):
    aux_event = {}
    aux_event["event_team"] = event["class"][-1][-4:]
    aux_event["event_time"] = event.find(
        "div", class_="match-events__item-timeline"
    ).text

    if aux_event["event_time"] == " PK ":
        aux_event["event_time"] = False
        aux_event["event_type"] = "PK"
        aux_event["event_result"] = event.find("img", class_="of-image__img")["alt"]
        aux_event["event_player"] = (
            event.find("div", class_="match-events__text").find("p").text
        )
    else:
        try:
            aux_event["event_type"] = event.find(class_="match-events__icon")[
                "aria-label"
            ]
        except:
            aux_event["event_type"] = event.find("img", class_="of-image__img")["alt"]
        try:
            for i, text in enumerate(
                event.find("div", class_="match-events__text").find_all("p")
            ):
                aux_event["action_player_" + str(i + 1)] = text.text
        except:
            aux_event["action_player_1"] = event.find(
                "p", class_="match-events__text"
            ).text

    return aux_event


def get_info_from_match(page, second_lineup):
    aux_dict = {}

    ## NAMES
    aux_dict["team_name_home"] = page.find(
        "span", class_="match-score-team__name--home"
    ).text
    aux_dict["team_name_away"] = page.find(
        "span", class_="match-score-team__name--away"
    ).text

    ## GOAL
    scores = page.find("p", class_="match-score-scores").find_all("span")
    aux_dict["team_home_score"] = scores[0].text
    aux_dict["team_away_score"] = scores[2].text

    ## STATISTICS
    description = page.find_all("div", class_="match-stats__stat-description")
    for d in description:
        field = format_field(d.find_all("p")[1].text)
        aux_dict[field + "_home"] = d.find_all("p")[0].text
        aux_dict[field + "_away"] = d.find_all("p")[2].text
        
    ## PREDICTION
    predictions = page.find('ul',class_='match-prediction__buttons').find_all('li')
    aux_dict['prediction_team_home_win'] = predictions[0].text
    aux_dict['prediction_draw'] = predictions[1].text
    aux_dict['prediction_team_away_win'] = predictions[2].text

    print(len(predictions))
        
    ## LOCATION
    entries = page.find('ul',class_='match-info__entries').find_all('li')
    aux_dict["location"] = entries[-1].find('span', class_='title-8-regular match-info__entry-subtitle').text

    ## EVENTS
    events = page.find("ul", class_="match-events__list").find_all(
        "li", class_="match-events__item"
    )
    aux_dict["events_list"] = [get_info_from_event(event) for event in events]

    aux_dict["lineup_home"] = get_lineups(page)
    aux_dict["lineup_away"] = get_lineups(second_lineup)

    return aux_dict


def get_lineups(page):
    lineup = []
    for player in page.find_all("div", class_="match-lineup-v2__player"):
        player_info = player.find('p',class_='match-lineup-v2__player-name').text.split('.')
        
        lineup.append(
            {
                 "player_name": player_info[1],
                "player_number": player_info[0],
            }
        )
    return lineup



def get_penaltis(match):
    regex_rule = r"\((\d+)\)"
    aux = match.find_all(
        "span", class_="title-7-bold simple-match-card-team__score"
    )
    aux = [
        re.search(regex_rule, i.text)[0].replace("(", "").replace(")", "") for i in aux
    ]
    return aux


def get_all_matches(html_full):
    aux_dict = []
    for match in html_full.find_all("a", class_="match-card", href=True):
        aux_dict_2 = {}
        aux_dict_2["link"] = "https://onefootball.com" + match["href"]
        aux_dict_2["stage"] = match.find_previous(
            "p", class_="label simple-match-cards-list__title-deprecated"
        ).text
        try:
            aux_dict_2["date"] = match.find("time").text
            aux_dict_2["pens"] = False
            aux_dict_2["pens_home_score"] = False
            aux_dict_2["pens_away_score"] = False
        except:
            aux_dict_2["date"] = False
            aux_dict_2["pens"] = (
                match.find(
                    "span",
                    class_="title-8-medium simple-match-card__info-message simple-match-card__info-message--secondary",
                ).text
                == "(Pens)"
            )
            home, away = get_penaltis(match)
            aux_dict_2["pens_home_score"] = home
            aux_dict_2["pens_away_score"] = away

        aux_dict.append(aux_dict_2)
    return aux_dict


<IPython.core.display.Javascript object>

## Competition

In [31]:
url = "https://onefootball.com/en/competition/olympic-games-53/results"  ## Competition link
page = get_html_full(url)
matchs = get_all_matches(page)
matchs[0]

{'link': 'https://onefootball.com/en/match/2210681',
 'stage': ' Final ',
 'date': ' 07.08.2021 ',
 'pens': False,
 'pens_home_score': False,
 'pens_away_score': False}

<IPython.core.display.Javascript object>

In [32]:
%%time

def merge_information(match):
    match_info = get_info_from_match(*get_html_from_a_match(match["link"]))
    return{**match, **match_info}
    
from joblib import Parallel, delayed
aux_list = Parallel(n_jobs=6, verbose=10)(delayed(merge_information)(match) for match in matchs)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:   16.8s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:   19.3s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:   46.4s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:   59.8s
[Parallel(n_jobs=6)]: Done  25 out of  32 | elapsed:  1.2min remaining:   20.4s
[Parallel(n_jobs=6)]: Done  29 out of  32 | elapsed:  1.3min remaining:    8.1s


CPU times: user 135 ms, sys: 77.9 ms, total: 213 ms
Wall time: 1min 23s


[Parallel(n_jobs=6)]: Done  32 out of  32 | elapsed:  1.4min finished


<IPython.core.display.Javascript object>

In [33]:
df = pd.DataFrame(aux_list).drop("link", axis=1)
df.shape

(32, 24)

<IPython.core.display.Javascript object>

## Manual adjustment

In [34]:
df[df.date == False]

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,...,shots_on_target_away,duels_won_home,duels_won_away,prediction_team_home_win,prediction_draw,prediction_team_away_win,location,events_list,lineup_home,lineup_away
3,Semi-finals,False,True,1,4,Mexico U23,Brazil U23,0,0,32%,...,6,50%,50%,54%,2%,44%,Kashima Stadium,"[{'event_team': 'away', 'event_time': ' 37' ',...","[{'player_name': ' Martín Mex ', 'player_numbe...","[{'player_name': ' Richarlison ', 'player_numb..."
6,Quarter-finals,False,True,4,2,Japan U23,New Zealand U23,0,0,49%,...,3,47%,53%,83%,3%,14%,Kashima Stadium,"[{'event_team': 'away', 'event_time': ' 19' ',...","[{'player_name': ' Hayashi ', 'player_number':...","[{'player_name': ' Waine ', 'player_number': '..."


<IPython.core.display.Javascript object>

In [36]:
df.date.iloc[3] = "03.08.2021"  ## Mexico U23xBrazil U23
df.date.iloc[6] = "31.07.2021"  ## Japan U23xNew Zealand U23

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


<IPython.core.display.Javascript object>

In [37]:
df

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,...,shots_on_target_away,duels_won_home,duels_won_away,prediction_team_home_win,prediction_draw,prediction_team_away_win,location,events_list,lineup_home,lineup_away
0,Final,07.08.2021,False,False,False,Brazil U23,Spain U23,2,1,41%,...,3,54%,46%,54%,2%,44%,"Nissan Stadium, Yokohama","[{'event_team': 'home', 'event_time': ' 20' ',...","[{'player_name': ' Richarlison ', 'player_numb...","[{'player_name': ' Olmo Carvajal ', 'player_nu..."
1,Match for third place,06.08.2021,False,False,False,Mexico U23,Japan U23,3,1,42%,...,6,50%,51%,68%,2%,30%,Saitama Stadium 2002,"[{'event_team': 'home', 'event_time': ' 13' ',...","[{'player_name': ' Martín Mex ', 'player_numbe...","[{'player_name': ' Hayashi ', 'player_number':..."
2,Semi-finals,03.08.2021,False,False,False,Japan U23,Spain U23,0,1,31%,...,6,58%,42%,19%,2%,79%,Saitama Stadium 2002,"[{'event_team': 'away', 'event_time': ' 13' ',...","[{'player_name': ' Hayashi ', 'player_number':...","[{'player_name': ' Olmo Carvajal ', 'player_nu..."
3,Semi-finals,03.08.2021,True,1,4,Mexico U23,Brazil U23,0,0,32%,...,6,50%,50%,54%,2%,44%,Kashima Stadium,"[{'event_team': 'away', 'event_time': ' 37' ',...","[{'player_name': ' Martín Mex ', 'player_numbe...","[{'player_name': ' Richarlison ', 'player_numb..."
4,Quarter-finals,31.07.2021,False,False,False,Korea Republic U23,Mexico U23,3,6,56%,...,9,48%,52%,13%,2%,85%,"Nissan Stadium, Yokohama","[{'event_team': 'away', 'event_time': ' 12' ',...","[{'player_name': ' Hwang ', 'player_number': '...","[{'player_name': ' Vega ', 'player_number': ' ..."
5,Quarter-finals,31.07.2021,False,False,False,Brazil U23,Egypt U23,1,0,64%,...,2,55%,46%,89%,2%,9%,Saitama Stadium 2002,"[{'event_team': 'home', 'event_time': ' 32' ',...","[{'player_name': ' Carneiro da Cunha ', 'playe...","[{'player_name': ' Mahmoud ', 'player_number':..."
6,Quarter-finals,31.07.2021,True,4,2,Japan U23,New Zealand U23,0,0,49%,...,3,47%,53%,83%,3%,14%,Kashima Stadium,"[{'event_team': 'away', 'event_time': ' 19' ',...","[{'player_name': ' Hayashi ', 'player_number':...","[{'player_name': ' Waine ', 'player_number': '..."
7,Quarter-finals,31.07.2021,False,False,False,Spain U23,Côte d'Ivoire U23,5,2,71%,...,6,55%,45%,87%,2%,11%,Miyagi Stadium,"[{'event_team': 'away', 'event_time': ' 10' ',...","[{'player_name': ' Olmo Carvajal ', 'player_nu...","[{'player_name': ' Dao ', 'player_number': ' 9..."
8,Round 3,28.07.2021,False,False,False,South Africa U23,Mexico U23,0,3,56%,...,6,48%,52%,9%,1%,90%,Sapporo Dome,"[{'event_team': 'home', 'event_time': ' 4' ', ...","[{'player_name': ' Makgopa ', 'player_number':...","[{'player_name': ' Vega ', 'player_number': ' ..."
9,Round 3,28.07.2021,False,False,False,France U23,Japan U23,0,4,47%,...,8,55%,46%,64%,4%,32%,"Nissan Stadium, Yokohama","[{'event_team': 'away', 'event_time': ' 22' ',...","[{'player_name': ' Kolo Muani ', 'player_numbe...","[{'player_name': ' Ueda ', 'player_number': ' ..."


<IPython.core.display.Javascript object>

In [38]:
df.to_csv("olympic_games_football_men_tokyo_2020-2021.csv", index=False)

<IPython.core.display.Javascript object>